In [1]:
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pandas as pd
import joblib

In [14]:
#texts = pd.read_parquet('datos.parquet')
df = pd.read_csv("textos ai c.csv")
texts = df["Text"].tolist()

In [15]:
texts

['El gato está durmiendo en el sofá ',
 'El perro juega en el parque',
 'La comida está lista en la mesa',
 'El sol brilla en el cielo azul',
 'Los pájaros cantan en el árbol',
 'El día amaneció soleado y los pájaros cantaban alegremente en los árboles',
 'La brisa fresca del mar acariciaba suavemente el rostro de quienes paseaban por la costa',
 'Los niños correteaban y jugaban en el parque',
 'La lluvia caía con suavidad',
 'El aroma del café recién hecho invadía la cocina',
 'Los libros en la estantería invitaban a sumergirse en mundos imaginarios y aprender nuevos conocimientos',
 'El sonido del piano resonaba en la sala',
 'Las montañas se alzaban majestuosas en el horizonte',
 'Las olas del mar rompían con fuerza contra las rocas',
 'El silencio de la noche era interrumpido solo por el canto de los grillos y el susurro del viento',
 'Las luces de la ciudad brillaban en la oscuridad',
 'Las risas de los niños en el parque infantil alegraban el corazón de quienes los escuchaban',
 

In [16]:
# Tokenizar el texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding para igualar la longitud de las secuencias
#maxlen = max(len(seq) for seq in sequences)
maxlen = 100
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

# Datos de entrada y salida
x_train = padded_sequences[:, :-1]  # Todas las columnas excepto la última
y_train = padded_sequences[:, -1]   # Última columna

In [17]:
# Tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1

# Modelo de lenguaje
embedding_dim = 50
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen-1),
    LSTM(100),
    Dense(vocab_size, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\marko\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
model.summary()

# Compilar y entrenar el modelo
model.fit(x_train, y_train, batch_size=30 ,epochs=100)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (30, 99, 50)           │        48,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (30, 100)              │        60,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (30, 972)              │        98,172 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 621,518 (2.37 MB)

 Trainable params: 207,172 (809.27 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 414,346 (1.58 MB)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9470 - loss: 0.8689
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9539 - loss: 0.8451
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9567 - loss: 0.8453
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9620 - loss: 0.8149
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9711 - loss: 0.8005
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9585 - loss: 0.8126
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9686 - loss: 0.8003
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9653 - loss: 0.7643
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9623 - loss: 0.7611
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9715 - loss: 0.7470
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9799 - loss: 0.7117
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9765 - lo

In [38]:
def generate_text(model, tokenizer, input_text, maxlen, num_words):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=maxlen-1)

    generated_sentence = input_text
    for _ in range(num_words):
        predicted_probabilities = model.predict(input_seq, verbose=0)
        predicted_word_idx = np.random.choice(len(predicted_probabilities[0]), p=predicted_probabilities[0])
        generated_word = tokenizer.index_word.get(predicted_word_idx, "")

        if generated_word:
            generated_sentence += " " + generated_word
            input_seq = np.append(input_seq[:, 1:], predicted_word_idx)
            input_seq = input_seq.reshape(1, -1)

    return generated_sentence

input_text = "la casa esta pintada de color"
generated_text = generate_text(model, tokenizer, input_text, maxlen, num_words=3)
print("Texto generado:", generated_text)

Texto generado: la casa esta pintada de color blanco compañero blanco


In [33]:
# Guardar el modelo existente
model.save('modelo/modelo_text.h5')

In [28]:
joblib.dump(tokenizer, 'modelo/tokenizer.pkl')

['modelo/tokenizer.pkl']